<h2>Moto GP ETL</h2>

In [1]:
import kagglehub
import pandas as pd
from backend.models.Season import Season, Race
from backend.utils.database import SessionLocal

/home/pasanca_7/Escritorio/worksapce/MotoGP_API/.env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Download dataset

In [3]:
dataset_path = '/home/pasanca_7/.cache/kagglehub/datasets/amalsalilan/motogpresultdataset/versions/1/'
try:
    df = pd.read_csv(dataset_path+"/FILTERED_ROWS.csv")
except FileNotFoundError:
    dataset_path = kagglehub.dataset_download("amalsalilan/motogpresultdataset")
    df = pd.read_csv(dataset_path+"/FILTERED_ROWS.csv")


Create database connection

In [4]:
db = SessionLocal()

<h4>Load 2021 season data</h4>

In [5]:
season21 = Season(
    year=2021
)
db.add(season21)

In [11]:
df21 = df[(df["year"] == 2021) & (df["category"] == "MotoGP")]
races_df = df21.drop_duplicates(subset=["sequence"], keep="first")
races_df = races_df[["sequence","shortname", "circuit_name"]]

<h3>Races</h3>

In [12]:
races = []
for _, row in races_df.iterrows():
    race = Race(
        sequence=row["sequence"],
        short_name=row["shortname"],
        circuit_name=row["circuit_name"],
        season=season21
    )
    races.append(race)
db.add_all(races)

NameError: name 'season21' is not defined

<h3>Teams</h3>

In [13]:
teams = df21[["team_name","bike_name"]].drop_duplicates(subset=["team_name","bike_name"])

<h3>Riders</h3>

In [31]:
riders = df21[["rider", "rider_name", "number", "country", "team_name"]].drop_duplicates(subset=["rider"])

In [29]:
contracts = df21.drop_duplicates(subset=(["rider"]))

,rider,rider_name,number,country,team_name
55025,2428,"Viñales, Maverick",12.0,ES,Monster Energy Yamaha MotoGP
55026,2330,"Zarco, Johann",5.0,FR,Pramac Racing
55027,2499,"Bagnaia, Francesco",63.0,IT,Ducati Lenovo Team
55028,2585,"Mir, Joan",36.0,ES,Team SUZUKI ECSTAR
55029,2563,"Quartararo, Fabio",20.0,FR,Monster Energy Yamaha MotoGP
55030,2471,"Rins, Alex",42.0,ES,Team SUZUKI ECSTAR
55031,2152,"Espargaro, Aleix",41.0,ES,Aprilia Racing Team Gresini
55032,2218,"Espargaro, Pol",44.0,ES,Repsol Honda Team
55033,2445,"Miller, Jack",43.0,AU,Ducati Lenovo Team
55034,2535,"Bastianini, Enea",23.0,IT,Esponsorama Racing


In [ ]:
db.commit()
db.close()